In [1]:
import pandas as pd

DATA = '/kaggle/input/predict-bankruptcy-in-poland/data.csv'

df = pd.read_csv(filepath_or_buffer=DATA)
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A57,A58,A59,A60,A61,A62,A63,A64,class,year
0,0.200550,0.37951,0.39641,2.0472,32.3510,0.38825,0.249760,1.33050,1.1389,0.50494,...,0.39718,0.87804,0.001924,8.4160,5.1372,82.658,4.4158,7.4277,0,1
1,0.209120,0.49988,0.47225,1.9447,14.7860,0.00000,0.258340,0.99601,1.6996,0.49788,...,0.42002,0.85300,0.000000,4.1486,3.2732,107.350,3.4000,60.9870,0,1
2,0.248660,0.69592,0.26713,1.5548,-1.1523,0.00000,0.309060,0.43695,1.3090,0.30408,...,0.81774,0.76599,0.694840,4.9909,3.9510,134.270,2.7185,5.2078,0,1
3,0.081483,0.30734,0.45879,2.4928,51.9520,0.14988,0.092704,1.86610,1.0571,0.57353,...,0.14207,0.94598,0.000000,4.5746,3.6147,86.435,4.2228,5.5497,0,1
4,0.187320,0.61323,0.22960,1.4063,-7.3128,0.18732,0.187320,0.63070,1.1559,0.38677,...,0.48431,0.86515,0.124440,6.3985,4.3158,127.210,2.8692,7.8980,0,1


We expect bankruptcies to be rare, so we expect our target class to be unbalanced.

In [2]:
df['class'].value_counts(normalize=True).to_dict()

{0: 0.9518258265176823, 1: 0.04817417348231771}

Only about 5% of our instances are bankruptcies.

Let's look at the distribution of cases across the years.

In [3]:
from plotly import express
import warnings

warnings.filterwarnings(action='ignore', category=FutureWarning)
express.histogram(data_frame=df, x='year', color='class', facet_col='class')

Our cases are more normally than unformly distributed across the years.